In [22]:
import os
%pwd

'D:\\Deep Learning\\Tensorflow & Keras\\deploy2\\project3'

In [23]:
os.chdir(r'D:\Deep Learning\Tensorflow & Keras\deploy2\project3')

In [24]:
%pwd

'D:\\Deep Learning\\Tensorflow & Keras\\deploy2\\project3'

In [25]:
from dataclasses import dataclass
from pathlib import Path
from TomatoDiseases.constant import *
from TomatoDiseases.utils.common import *

In [26]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [27]:
class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params  = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= self.config.training.trained_model_path,
            training_data= self.config.data_ingestion.unzip_dir,
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        
        return eval_config

In [28]:
import tensorflow as tf

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def valid_generator(self):
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.3
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= 'validation',
            shuffle=False,
            **dataflow_kwargs
        ) 
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluate(self):
        model = tf.keras.models.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {'loss': self.score[0], 'recall': self.score[1]}
        save_json(path=Path('scores.json'), data=scores)

In [29]:
try:
    config = ConfigManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluate()
    evaluation.save_score()
except Exception as e:
    raise e

[2023-07-12 00:03:03,755: INFO: common: yaml file: config.yaml loaded successfully]
[2023-07-12 00:03:03,758: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-12 00:03:03,759: INFO: common: created directory at: artifacts]
Found 5500 images belonging to 10 classes.
  7/344 [..............................] - ETA: 5:57 - loss: 0.7628 - recall: 0.9554

KeyboardInterrupt: 